In [ ]:
import polars as pl
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [2]:
label2id = {"O": 0, "B-ASP": 1, "I-ASP": 2}
id2label = {v: k for k, v in label2id.items()}

In [7]:
df=pl.read_parquet("../data/processed/df_aspect_pos.parquet")

In [4]:
df = df.sample(50)

In [8]:
df.columns

['input_ids',
 'attention_mask',
 'labels',
 'aspects_index',
 'aspects_sentiment',
 'type']

In [ ]:
def custom_collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])

    aspects_index = [item["aspects_index"] for item in batch]          # List[List[List[int]]]
    aspects_sentiment = [item["aspects_sentiment"] for item in batch]  # List[List[int]]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "aspects_index": aspects_index,
        "aspects_sentiment": aspects_sentiment,
    }


In [11]:
def extract_aspect_spans(pred_labels):
    spans = []
    i = 0
    while i < len(pred_labels):
        if pred_labels[i] == 1:  # B-ASP
            start = i
            i += 1
            while i < len(pred_labels) and pred_labels[i] == 2:  # I-ASP
                i += 1
            end = i - 1
            spans.append([start, end])
        else:
            i += 1
    return spans

In [12]:
class CustomDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df["input_ids"].to_numpy()
        self.attention_mask = df["attention_mask"].to_numpy()
        self.labels = df["labels"].to_numpy()
        self.aspects_index = df["aspects_index"].to_list()
        self.aspects_sentiment = df["aspects_sentiment"].to_list()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx], dtype=torch.long),
            "attention_mask": torch.tensor(self.attention_mask[idx], dtype=torch.long),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
            "aspects_index": self.aspects_index[idx],           # list of [start, end]
            "aspects_sentiment": self.aspects_sentiment[idx],   # list of sentiment values
        }


batch_size = 32
train_dataset = CustomDataset(df.filter(pl.col("type") == "train"))
val_dataset = CustomDataset(df.filter(pl.col("type") == "val"))
test_dataset = CustomDataset(df.filter(pl.col("type") == "test"))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size , collate_fn=custom_collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=custom_collate_fn)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)


In [13]:
for batch in train_dataloader:
    print(batch["input_ids"].shape)
    print(batch["attention_mask"].shape)
    print(batch["labels"].shape)
    print(batch["aspects_sentiment"])
    print(batch["aspects_index"])
    break

torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32, 128])
[[1], [1], [0], [1], [1], [1], [0, 1], [1], [0], [1, 1, 1], [1], [0], [2], [1, 1], [1, 1, 1], [1, 1, 1], [1], [1], [0], [1], [1, 1], [1], [1], [1], [1], [0], [1], [1], [1], [1, 1], [1], [1]]
[[[2, 2]], [[15, 15]], [[7, 7]], [[2, 3]], [[13, 16]], [[4, 4]], [[2, 3], [7, 8]], [[7, 7]], [[2, 4]], [[7, 7], [10, 10], [15, 15]], [[2, 3]], [[11, 11]], [[6, 8]], [[2, 2], [8, 9]], [[1, 9], [18, 19], [39, 40]], [[2, 2], [5, 6], [9, 9]], [[13, 13]], [[2, 3]], [[6, 7]], [[8, 10]], [[3, 4], [11, 14]], [[2, 2]], [[4, 8]], [[2, 2]], [[2, 2]], [[5, 5]], [[2, 2]], [[1, 1]], [[2, 3]], [[2, 2], [17, 17]], [[2, 2]], [[4, 5]]]


In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim=hidden_size, num_heads=1, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.norm = nn.LayerNorm(hidden_size)
        self.classifier = nn.Linear(hidden_size, 3)  # Sentiment classes: pos, neg, neutral

    def forward(self, token_embeddings, aspect_mask):
        # token_embeddings: [B, L, H], aspect_mask: [B, L]
        aspect_mask = aspect_mask.unsqueeze(-1).expand_as(token_embeddings)  # [B, L, H]
        aspect_embeddings = token_embeddings * aspect_mask  # Zero out non-aspect tokens

        aspect_pooled = aspect_embeddings.sum(dim=1) / (aspect_mask.sum(dim=1) + 1e-8)  # [B, H]

        query = aspect_pooled.unsqueeze(1)  # [B, 1, H]
        key = value = token_embeddings  # [B, L, H]

        attended_output, attn_weights = self.attention(query, key, value)  # [B, 1, H]
        attended_output = self.dropout(attended_output)
        attended_output = self.norm(attended_output)

        logits = self.classifier(attended_output.squeeze(1))  # [B, 3]
        return logits, attn_weights 


class AspectDetectionModel(nn.Module):
    def __init__(self):
        super(AspectDetectionModel, self).__init__()
        self.bert = AutoModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.token_classifier = nn.Linear(self.bert.config.hidden_size, len(label2id))
        self.sentiment_classifier = SentimentClassifier(hidden_size=self.bert.config.hidden_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs.last_hidden_state)  # [B, L, H]

        token_logits = self.token_classifier(sequence_output)  # For aspect term tagging (BIO)

        return token_logits, sequence_output

model = AspectDetectionModel().to("mps")
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)

In [25]:
num_epochs = 20

for epoch in range(num_epochs):
    total_aspect_train_loss = 0
    total_sentiment_train_loss = 0
    total_aspect_val_loss = 0
    total_sentiment_val_loss = 0

    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to("mps")
        attention_mask = batch["attention_mask"].to("mps")
        labels = batch["labels"].to("mps")


        token_logits, sequence_output = model(input_ids, attention_mask)
        aspect_loss = criterion(token_logits.view(-1, len(label2id)), labels.view(-1))
        total_aspect_train_loss += aspect_loss.item()


      
        sentiment_losses = []
        for i in range(len(input_ids)):
            for aspect_index, sentiment in zip(batch["aspects_index"][i], batch["aspects_sentiment"][i]):
                if aspect_index[1] >= sequence_output.size(1):
                    continue
                # Create a new aspect mask for each aspect instead of modifying in-place
                aspect_mask = torch.zeros_like(input_ids, dtype=torch.float).to("mps")
                aspect_mask[i, aspect_index[0]:aspect_index[1]+1] = 1
                sentiment_logits, _ = model.sentiment_classifier(sequence_output[i].unsqueeze(0), aspect_mask[i].unsqueeze(0))
                sentiment_target = torch.tensor([sentiment], dtype=torch.long).to("mps")
                sentiment_loss = criterion(sentiment_logits, sentiment_target)
                sentiment_losses.append(sentiment_loss)


        if sentiment_losses:
            sentiment_loss = torch.stack(sentiment_losses).mean()
        else:
            sentiment_loss = torch.tensor(0.0).to("mps")

        total_sentiment_train_loss += sentiment_loss.item()
        total_loss = aspect_loss + sentiment_loss

        total_loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
        model.eval()

        for batch in val_dataloader:
            input_ids = batch["input_ids"].to("mps")
            attention_mask = batch["attention_mask"].to("mps")
            labels = batch["labels"].to("mps")

            token_logits, sequence_output = model(input_ids, attention_mask)
            aspect_loss = criterion(token_logits.view(-1, len(label2id)), labels.view(-1))
            total_aspect_val_loss += aspect_loss.item()

            sentiment_losses = []
            preds = torch.argmax(token_logits, dim=2)
            for i in range(len(input_ids)):
                aspects = extract_aspect_spans(preds[i].cpu().tolist())
                for aspect_index, sentiment in zip(batch["aspects_index"][i], batch["aspects_sentiment"][i]):
                    if aspect_index in aspects and aspect_index[1] < sequence_output.size(1):
                        aspect_mask = torch.zeros_like(input_ids, dtype=torch.float).to("mps")
                        aspect_mask[i, aspect_index[0]:aspect_index[1]+1] = 1
                        sentiment_logits, _ = model.sentiment_classifier(sequence_output[i].unsqueeze(0), aspect_mask[i].unsqueeze(0))
                        sentiment_target = torch.tensor([sentiment], dtype=torch.long).to("mps")
                        sentiment_loss = criterion(sentiment_logits.view(-1, 3), sentiment_target)
                        sentiment_losses.append(sentiment_loss)

            if sentiment_losses:
                sentiment_loss = torch.stack(sentiment_losses).mean()
            else:
                sentiment_loss = torch.tensor(0.0).to("mps")

            total_sentiment_val_loss += sentiment_loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], "
        f"Train Aspect Loss: {total_aspect_train_loss/len(train_dataloader):.4f}, "
        f"Train Sentiment Loss: {total_sentiment_train_loss/len(train_dataloader):.4f}, "
        f"Val Aspect Loss: {total_aspect_val_loss/len(val_dataloader):.4f}, "
        f"Val Sentiment Loss: {total_sentiment_val_loss/len(val_dataloader):.4f}")



Epoch [1/20], Train Aspect Loss: 0.3742, Train Sentiment Loss: 0.5146, Val Aspect Loss: 0.2219, Val Sentiment Loss: 0.3944
Epoch [2/20], Train Aspect Loss: 0.1975, Train Sentiment Loss: 0.3532, Val Aspect Loss: 0.1700, Val Sentiment Loss: 0.3869
Epoch [3/20], Train Aspect Loss: 0.1503, Train Sentiment Loss: 0.2818, Val Aspect Loss: 0.1405, Val Sentiment Loss: 0.4164
Epoch [4/20], Train Aspect Loss: 0.1240, Train Sentiment Loss: 0.2235, Val Aspect Loss: 0.1275, Val Sentiment Loss: 0.3928
Epoch [5/20], Train Aspect Loss: 0.1118, Train Sentiment Loss: 0.1878, Val Aspect Loss: 0.1218, Val Sentiment Loss: 0.4012
Epoch [6/20], Train Aspect Loss: 0.1021, Train Sentiment Loss: 0.1619, Val Aspect Loss: 0.1172, Val Sentiment Loss: 0.4071
Epoch [7/20], Train Aspect Loss: 0.0930, Train Sentiment Loss: 0.1262, Val Aspect Loss: 0.1168, Val Sentiment Loss: 0.4575
Epoch [8/20], Train Aspect Loss: 0.0867, Train Sentiment Loss: 0.1029, Val Aspect Loss: 0.1146, Val Sentiment Loss: 0.4285
Epoch [9/20], Tr

In [ ]:
import os
name="AspectDetectionModel_Sentiment_Analysis_Attention"

os.makedirs("../models/AspectDetectionModel", exist_ok=True)

torch.save(model.state_dict(), "../models/AspectDetectionModel/" + name + ".pth")

model_config = {
	"hidden_size": model.bert.config.hidden_size,
	"num_labels": len(label2id),
	"id2label": id2label,
	"label2id": label2id,
	"name": name
}

import json
with open(f"../models/AspectDetectionModel/{name}_config.json", "w") as f:
	json.dump(model_config, f)

print("Model saved successfully to ../models/AspectDetectionModel/")

Model saved successfully to ../models/AspectDetectionModel/


In [ ]:
name="AspectDetectionModel_Sentiment_Analysis"
model = AspectDetectionModel()
model.load_state_dict(torch.load("../models/AspectDetectionModel/" + name + ".pth"))
model = model.to("mps")
model.eval()

AspectDetectionModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [45]:
temp = tokenizer(("""Car quality is very nice but the controller sucks . The controller of this car do not works properly and the final in the controller do not rotate fully it only rotate like button""").split(), is_split_into_words=True,
                          truncation=True,
                          padding="max_length",
                          max_length=128)

print(temp)

{'input_ids': [101, 2482, 3737, 2003, 2200, 3835, 2021, 1996, 11486, 19237, 1012, 1996, 11486, 1997, 2023, 2482, 2079, 2025, 2573, 7919, 1998, 1996, 2345, 1999, 1996, 11486, 2079, 2025, 24357, 3929, 2009, 2069, 24357, 2066, 6462, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [46]:
device="mps"
model.eval()
with torch.no_grad():

    
    input_ids = torch.tensor(temp["input_ids"], dtype=torch.long).unsqueeze(0).to(device)
    attention_mask = torch.tensor(temp["attention_mask"], dtype=torch.long).unsqueeze(0).to(device)

    logits, sequence_output = model(input_ids, attention_mask)
    # decode the logits to get the predicted labels
    preds = torch.argmax(logits, dim=2)[0]
    aspects = extract_aspect_spans(preds.cpu().tolist())
    sentiments=[]
    for aspect in aspects:
        aspect_mask = torch.zeros_like(input_ids, dtype=torch.long).to("mps")
        aspect_mask[0, aspect[0]:aspect[1]+1] = 1
        # print("am", aspect_mask[0].unsqueeze(0))
        sentiment_logits, _ = model.sentiment_classifier(
            sequence_output, aspect_mask
        )
        sentiments.append({"pos":aspect,"senti":torch.argmax(sentiment_logits, dim=1).item()})

preds

tensor([0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='mps:0')

In [47]:
logits

tensor([[[ 4.5587, -2.3159, -2.7290],
         [ 0.2546,  1.6434, -2.5836],
         [ 0.9176, -1.5676,  0.9227],
         [ 4.8864, -2.4453, -2.7059],
         [ 5.2138, -2.4290, -3.0259],
         [ 5.0780, -2.0911, -2.9120],
         [ 5.2390, -2.5481, -3.3259],
         [ 5.2350, -2.8260, -3.0732],
         [-0.1817,  2.9009, -2.4859],
         [ 5.2310, -2.3744, -2.7506],
         [ 6.3429, -3.2683, -3.3029],
         [ 4.1683, -1.5477, -2.7579],
         [-0.6483,  3.1019, -2.4083],
         [ 2.2763, -2.9142,  0.4111],
         [ 5.5308, -2.7043, -2.8574],
         [ 2.8816, -0.6622, -3.0009],
         [ 5.4649, -2.4530, -2.8229],
         [ 5.4646, -2.3330, -2.7968],
         [ 5.4757, -2.2137, -3.1635],
         [ 5.4349, -2.2264, -3.1097],
         [ 5.2646, -2.4777, -2.9180],
         [ 4.5705, -2.3099, -2.4537],
         [ 2.7997, -0.4281, -2.0433],
         [ 3.4136, -1.8345, -0.6538],
         [ 4.3592, -3.6476, -1.4186],
         [ 1.5128,  0.3037, -2.6704],
         [ 5

In [48]:
arr=[-1] * 128
arr

for item in sentiments:
    pos=item["pos"]
    start = pos[0]
    end = pos[0]+pos[1]
    arr[start:end] = [item["senti"]] * (end-start)

In [49]:
aspects

[[1, 2], [8, 8], [12, 12], [67, 67], [80, 80], [95, 95], [96, 96]]

In [50]:
id2label[1]

'B-ASP'

In [51]:
predictions = [id2label[label] for label in preds.tolist()]
predictions

['O',
 'B-ASP',
 'I-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-ASP',
 'B-ASP',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [52]:
for item in zip(predictions, temp["input_ids"], arr):
    word = tokenizer.convert_ids_to_tokens(item[1])
    if word in ['[CLS]', '[SEP]', '[PAD]']:
        # print(item[0])
        continue
    print(item[0],"--------" ,word, "------", item[2])

B-ASP -------- car ------ 1
I-ASP -------- quality ------ 1
O -------- is ------ -1
O -------- very ------ -1
O -------- nice ------ -1
O -------- but ------ -1
O -------- the ------ -1
B-ASP -------- controller ------ 0
O -------- sucks ------ 0
O -------- . ------ 0
O -------- the ------ 0
B-ASP -------- controller ------ 0
O -------- of ------ 0
O -------- this ------ 0
O -------- car ------ 0
O -------- do ------ 0
O -------- not ------ 0
O -------- works ------ 0
O -------- properly ------ 0
O -------- and ------ 0
O -------- the ------ 0
O -------- final ------ 0
O -------- in ------ 0
O -------- the ------ -1
O -------- controller ------ -1
O -------- do ------ -1
O -------- not ------ -1
O -------- rotate ------ -1
O -------- fully ------ -1
O -------- it ------ -1
O -------- only ------ -1
O -------- rotate ------ -1
O -------- like ------ -1
O -------- button ------ -1


In [133]:
def extract_aspects(input_ids, predictions, tokenizer):
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    aspects = []
    current_aspect = []
    current_position = []
    current_idx = 0

    for idx, (token, label) in enumerate(zip(tokens, predictions[0])):
        # Skip special tokens
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue
            
        label_tag = id2label.get(label, "O")
        
        if label_tag == "B-ASP":
            # Save previous aspect if exists
            if current_aspect:
                aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))
            # Start new aspect
            current_aspect = [token]
            current_position = [idx]
        elif label_tag == "I-ASP" and current_aspect:
            # Continue current aspect
            current_aspect.append(token)
            current_position.append(idx)
        else:
            # End previous aspect if exists
            if current_aspect:
                aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))
                current_aspect = []
                current_position = []

    # Catch any leftover aspect
    if current_aspect:
        aspects.append((" ".join(current_aspect), (current_position[0], current_position[-1])))

    # Clean up tokens (remove ##)
    clean_aspects = []
    for aspect, pos in aspects:
        # Handle WordPiece tokens (tokens that start with ##)
        cleaned = ""
        for word in aspect.split():
            if word.startswith("##"):
                cleaned += word[2:]  # Remove ## prefix
            else:
                if cleaned:
                    cleaned += " "
                cleaned += word
        clean_aspects.append((cleaned, pos))

    return clean_aspects


extracted_aspects = extract_aspects(input_ids, predictions, tokenizer)
extracted_aspects

[('chrome', (2, 2)), ('book', (3, 3))]